In [ ]:
#!pip install pyjarowinkler

In [ ]:
import glob

In [ ]:
!pwd

In [ ]:
from craft_text_detector import Craft
#output_dir = '/path_to/craft_output/'
cuda = False

In [ ]:
%cd path_to_images
image_filenames = glob.glob('images/*')
image_filenames

In [ ]:
craft = Craft(output_dir,
              crop_type='poly', 
              cuda=False, 
              export_extra=True, 
              refiner=True,
              text_threshold=0.7,
              link_threshold=0.4,
              low_text=0.4)

In [ ]:
%cd path_to_images

for image_filename in image_filenames: 
    craft.detect_text(image_filename);
    print(image_filename)

In [ ]:
# unload models from ram/gpu
craft.unload_craftnet_model()
craft.unload_refinenet_model()

In [ ]:
#!git clone https://github.com/clovaai/deep-text-recognition-benchmark
%cd deep-text-recognition-benchmark

In [ ]:
outputs = []


In [ ]:
crop_dir_names = glob.glob('path_to_craft_output/craft_output/*crops')

In [ ]:
for crop_dir_name in crop_dir_names:
    output = !CUDA_VISIBLE_DEVICES=0 /Users/iljarasin/opt/anaconda3/envs/YOUR_ENV/bin/python demo.py \
    --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --sensitive\
    --image_folder $crop_dir_name \
    --saved_model TPS-ResNet-BiLSTM-Attn-case-sensitive.pth
    print(crop_dir_name)
    outputs.append(output)

In [ ]:
outputs[2]

In [ ]:
len(outputs)

In [ ]:
from IPython.core.display import display, HTML
from PIL import Image
import base64
import io
import pandas as pd

appended_data = []

for output in outputs[10:30]:
    for ind, row in enumerate(output[output.index('image_path               \tpredicted_labels         \tconfidence score')+2:]):
        row = row.split('\t')
        filename = row[0].strip()
        label = row[1].strip()
        conf = row[2].strip()
        img = Image.open(filename)
        img_buffer = io.BytesIO()
        img.save(img_buffer, format="PNG")
        imgStr = base64.b64encode(img_buffer.getvalue()).decode("utf-8") 
        data = pd.DataFrame()
        data.loc[ind, 'img'] = '<img src="data:image/png;base64,{0:s}">'.format(imgStr)
        data.loc[ind, 'id'] = filename
        data.loc[ind, 'label'] = label
        data.loc[ind, 'conf'] = conf
        appended_data.append(data)

appended_data = pd.concat(appended_data)
html_all = appended_data.to_html(escape=False)


In [ ]:
display(HTML(html_all))

In [ ]:
import string
string.printable[:-6]

1. Brand (e.g. Viajero)
2. Color (e.g. red wine)
3. Grape (e.g. Cabernet Sauvignon)
4. Taste (e.g. dry)
5. Region (e.g. Pfalz)
6. Country(e.g. Germany)
7. Year (e.g. 2015)
8. Filling quantity (e.g. 750 ml)
9. Type of container/vessel (e.g. bottle, tetra pack)
10. Type (e.g. quality wine)
11. Description (e.g. german red wine made of Dornfelder grapes from Pfalz)
12. Alcohol strength (e.g. 12%)
13. Allergens (e.g. contains sulfites)
14. EAN

In [ ]:
output

In [ ]:
appended_data.head(10)

4. Fine - img (unwrap, rotate, black - white)
1. Preproc
2. Dict
3. Matching

4. Fine - img (unwrap, rotate, black - white)

In [ ]:
appended_data.sort_values('conf', ascending=False)

In [ ]:
import numpy as np
result = appended_data.copy()
col = result['id'].str.split('/')

In [ ]:
col = [np.array(l)[[6,7]] for l in col]

In [ ]:
col = ['-'.join(l) for l in col]

In [ ]:
result.id = col

In [ ]:
result.drop('img', axis=1, inplace=True)

In [ ]:
result.reset_index(inplace=True, drop=True)

In [ ]:
result.head(20)

In [ ]:
#result = result.sort_values('conf', ascending=False)

In [ ]:
result.dtypes

In [ ]:
result.conf = result.conf.astype(float)

In [ ]:
#result = result[result.conf > .4]
#result

In [ ]:
# evtl more steps required
result['label_cleanup'] = result['label'].str.lower()

In [ ]:
result.head(25)

#### Test Hamming Distance

In [ ]:

def hamming(str1, str2):

    score = 0

    for (a, b) in zip(str1, str2):
        a_bits = bin(ord(a))[2:].zfill(8)
        b_bits = bin(ord(b))[2:].zfill(8)

        score += sum(bx != by for bx, by in zip(a_bits, b_bits))
    return score

In [ ]:
# not sufficient accuracy
hamming('itl', 'Italy')

In [ ]:
#!pip install xlrd

In [ ]:
%cd path_to_glossars
rebsorten = pd.read_excel('Weinglossar.xlsx', sheet_name='Rebsorten')

In [ ]:
rebsorten = rebsorten[['Rebsorten']]

In [ ]:
rebsorten.dropna(axis=0, inplace=True)

In [ ]:
rebsorten.isna().any()

In [ ]:
rebsorten[0:10]

In [ ]:
weinanbaugebiet = pd.read_excel('Weinglossar.xlsx', sheet_name='Weinbaugebiet')

In [ ]:
weinanbaugebiet

In [ ]:
weinanbaugebiet.isna().any()

In [ ]:
glossary = pd.read_excel('glossary.xlsx', sheet_name='Sheet1', index_col='Unnamed: 0')

In [ ]:
glossary.tail(5)

In [ ]:
result.shape

In [ ]:
result['winery'] = ''
result['wine'] = ''
result['country'] = ''
result['alcohol strength'] = ''
# Filling quantity
result['filling quantity'] = ''
# Grape varieties
result['grape variety'] = ''
result['wine growing area'] = ''
result['allergens'] = ''
NOT_FOUND = '-'

#### Über verschiedene Distance-Types and Jaron-Winkler Distance

https://en.wikipedia.org/wiki/Jaro–Winkler_distance

In [ ]:
# good one!!
from pyjarowinkler import distance
distance.get_jaro_distance("enth", "enthält", winkler=True, scaling=0.1)

In [ ]:
result.head(5)

In [ ]:
# regex for percent \\d+(?:\\.\\d+)?%
# (\d+(\.\d+)?%)
regex_pecent = re.compile('(\d+(\.\d+)?%)') 


In [ ]:
#test regex
np.array(regex_pecent.findall('12%')[0][0])

In [ ]:
alk = [regex_pecent.findall(x)[0][0] if not len(regex_pecent.findall(x))==0 else NOT_FOUND for x in result.label_cleanup]

In [ ]:
np.array(alk)[0:50]

In [ ]:
result['alcohol strength'] = alk
result.head(5)

#### Test quantulum to extract measures

In [ ]:
#!pip install quantulum3

In [ ]:
# not good for filling quantity
from quantulum3 import parser
quants = parser.parse('eee250ml')

In [ ]:
quants

#### Bad one!

In [ ]:
#r"[0-9\.\s]+(?:mg|kg|ml|q.s.|ui|L|l|M|g|µg)"
regex_ml = re.compile( r"[0-9\.|,\s]+(?:ml|L|l)" )
x = '0,250L'
z = regex_ml.findall(x)
z

In [ ]:
quantity = [regex_ml.findall(x)[0] if not len(regex_ml.findall(x))==0 else NOT_FOUND for x in result.label_cleanup]

In [ ]:
result['filling quantity'] = quantity

In [ ]:
result.head(5)

In [ ]:
countries = np.unique(np.concatenate((glossary.country.unique(), weinanbaugebiet.Land.unique())))

In [ ]:
countries

In [ ]:

distance.get_jaro_distance("250", "enthält", winkler=True, scaling=0.1)

In [ ]:
distance.get_jaro_distance('max', 'maxii', winkler=True, scaling=0.1)

In [ ]:
def get_best_match(label, collection, threshold):
    found = NOT_FOUND
    distances = []
    for elem in collection:
        dist = distance.get_jaro_distance(label, elem, winkler=True, scaling=0.1)
        distances.append(dist)
        
    distances = np.array(distances)
    print(distances)
    dist = max(distances)
    print(dist, 'max likelyhood of match')
    
    if dist > threshold:
        index = np.argmax(distances)
        print(index, 'index of max distance')
        
        found = collection[index]
        print(found, 'is the matched element')
    return found
            

In [ ]:
get_best_match('fra', countries, .8)

In [ ]:
countries_matched = [get_best_match(x, countries, .7) for x in result.label_cleanup]

In [ ]:
countries_matched

In [ ]:
result.country = countries_matched

In [ ]:
result.head(50)

In [ ]:
glossary.dtypes

In [ ]:
glossary.Winery.unique()

In [ ]:
winery_matched = [get_best_match(x, glossary.Winery.unique(), .8) for x in result.label_cleanup]

In [ ]:
winery_matched

In [ ]:
result.winery = winery_matched
result.head(20)

In [ ]:
glossary.Wine = glossary.Wine.astype(str)

In [ ]:
wine_matched = [get_best_match(x, glossary.Wine.unique(), .8) for x in result.label_cleanup]

In [ ]:
wine_matched[0:20]

In [ ]:
result.wine = wine_matched

In [ ]:
result.head(10)

In [ ]:
grape_variety = [get_best_match(x, rebsorten.Rebsorten.unique(), .7) for x in result.label_cleanup]

In [ ]:
grape_variety

In [ ]:
result['grape variety'] = grape_variety

In [ ]:
result.head(20)

In [ ]:
wine_growing_area = [get_best_match(x, weinanbaugebiet.Weinbaugebiet.unique(), .8) for x in result.label_cleanup]

In [ ]:
result['wine growing area'] = wine_growing_area

In [ ]:
result.head(20)

In [ ]:
allergens = ['enthält Sulfite', 'enthält Schwefeldioxid', 'contains sulphites', 'contains sulphur dioxide']

#### We need it in all relevant languages
https://www.lgl.bayern.de/lebensmittel/warengruppen/wc_33_weine/et_kennzeichnung_sulfit_weine.htm

In [ ]:
allergens_match = [get_best_match(x, allergens, .7) for x in result.label_cleanup]

In [ ]:
result.allergens = allergens_match

In [ ]:
result.head(20)

In [ ]:
result[result.id.str.contains(pat = 'DSC_0044')] 